In [0]:
from pyspark.sql.functions import *

In [0]:
df_policy = spark.read.format("delta").load("/mnt/gold/monthly_policy_summary")
df_cases = spark.read.format("delta").load("/mnt/gold/covid_monthly_summary")

In [0]:
df_policy_effect = df_policy.join(df_cases, ["location", "year", "month"], "inner") 

In [0]:
df_policy_effect_2 = df_policy_effect.select(
        "location", "year", "month",
        "covid_new_cases", "covid_total_cases",
        "covid_new_deaths", "covid_total_deaths",
        "monthly_covid_case_rank", "monthly_covid_new_case_rank",
        "avg_school_close", "avg_work_close", "avg_stay_home",
        "avg_travel_control", "avg_test_policy", "avg_trace", "avg_stringency_index",
        "monthly_policy_strictness_rank", "monthly_travel_control_rank"
    )

In [0]:
df_policy_effect_3 = df_policy_effect_2 \
    .withColumn("high_stringency_flag", when(col("avg_stringency_index") > 70, 1).otherwise(0)) \
    .withColumn("strict_travel_control_flag", when(col("avg_travel_control") >= 3, 1).otherwise(0)) \
    .withColumn("potential_policy_effective",
                when((col("high_stringency_flag") == 1) & (col("monthly_covid_new_case_rank") >= 10), 1).otherwise(0))

In [0]:
df_policy_effect_3.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .save("/mnt/gold/policy_effect_on_covid")